In [ ]:
#!/bin/bash
!curl -L -o ~/content/cicddos2019.zip\
  https://www.kaggle.com/api/v1/datasets/download/dhoogla/cicddos2019

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 28.7M    0  1177    0     0   2114      0  3:57:37 --:--:--  3:57:37  2114
curl: (23) Failure writing output to destination


In [ ]:
!kaggle datasets download dhoogla/cicddos2019

Dataset URL: https://www.kaggle.com/datasets/dhoogla/cicddos2019
License(s): CC-BY-NC-SA-4.0
 63% 18.0M/28.7M [00:00<00:00, 52.3MB/s]
100% 28.7M/28.7M [00:00<00:00, 75.4MB/s]


In [ ]:
import zipfile
import os

# Define paths
zip_file_path = "/content/cicddos2019.zip"
extract_to_path = "/content/data/"

# Ensure the target directory exists
os.makedirs(extract_to_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Unzipping completed!")


Unzipping completed!


In [ ]:
import pandas as pd
import os
import glob

# Path to your Parquet files directory
path = "/content/data"  # Replace with the actual directory path

# List all parquet files
parquet_files = glob.glob(os.path.join(path, "*.parquet"))

# Concatenate all parquet files
dataframes = [pd.read_parquet(file) for file in parquet_files]
data = pd.concat(dataframes, ignore_index=True)

# Save the concatenated data to a new file
output_file = "data.parquet"
data.to_parquet(output_file, index=False)

print(f"All files concatenated and saved as {output_file}.")


All files concatenated and saved as data.parquet.


In [ ]:
!pip install zynet

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from zynet import Sequential, Dense, Dropout, softmax, compile_for_verilog

# Selected features based on the data you shared
selected_features = [
    'Protocol',
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Fwd Packets Length Total',
    'Bwd Packets Length Total',
    'Fwd Packet Length Max',
    'Fwd Packet Length Min',
    'Fwd Packet Length Mean',
    'Fwd Packet Length Std'
]

target_column = 'Label'

# Load data
data = pd.read_parquet("/content/data.parquet")
print("Data shape after concatenation:", data.shape)

# Encode the target variable
le_target = LabelEncoder()
data[target_column] = le_target.fit_transform(data[target_column])

# Split features and target
X = data[selected_features]
y = data[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build and train the neural network using ZyneT
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(le_target.classes_), activation=softmax)  # Multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Convert the trained model to Verilog code for FPGA/ASIC implementation
verilog_code = compile_for_verilog(model)

# Save or print the generated Verilog code
with open("neural_network.v", "w") as verilog_file:
    verilog_file.write(verilog_code)

print("Verilog code generated and saved as 'neural_network.v'")

# Extract weights and biases from the trained model
with open("WeightsAndBiases.txt", "w") as weights_file:
    weights_file.write("Weights and Biases of the Trained Neural Network:\n\n")

    for i, layer in enumerate(model.layers):
        if hasattr(layer, "weights"):  # Check if the layer has weights
            weights = layer.get_weights()

            if weights:
                weights_file.write(f"Layer {i + 1}: {layer.name}\n")

                if len(weights) > 0:
                    weights_file.write("Weights:\n")
                    weights_file.write(str(weights[0]))
                    weights_file.write("\n\n")

                if len(weights) > 1:
                    weights_file.write("Biases:\n")
                    weights_file.write(str(weights[1]))
                    weights_file.write("\n\n")

print("Weights and biases saved in 'WeightsAndBiases.txt'")



ModuleNotFoundError: No module named 'zynet'

In [ ]:
from zynet import zynet
from zynet import utils
import numpy as np

def genMnistZynet(dataWidth,sigmoidSize,weightIntSize,inputIntSize):
    model = zynet.model()
    model.add(zynet.layer("flatten",10))
    model.add(zynet.layer("Dense",30,"sigmoid"))
    model.add(zynet.layer("Dense",20,"sigmoid"))
    model.add(zynet.layer("Dense",10,"sigmoid"))
    model.add(zynet.layer("Dense",10,"hardmax"))
    weightArray = utils.genWeightArray('WeigntsAndBiases.txt')
    biasArray = utils.genBiasArray('WeigntsAndBiases.txt')
    model.compile(pretrained='Yes',weights=weightArray,biases=biasArray,dataWidth=dataWidth,weightIntSize=weightIntSize,inputIntSize=inputIntSize,sigmoidSize=sigmoidSize)
    zynet.makeXilinxProject('myProject1','xc7z020clg484-1')
    zynet.makeIP('myProject1')
    zynet.makeSystem('myProject1','myBlock2')

if __name__ == "__main__":
    genMnistZynet(dataWidth=8,sigmoidSize=10,weightIntSize=4,inputIntSize=1)

FileNotFoundError: [Errno 2] No such file or directory: 'WeigntsAndBiases.txt'

In [ ]:
data.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,1,2,0,422.0,0.0,211.0,211.0,211.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,17,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,17,1,2,0,458.0,0.0,229.0,229.0,229.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,17,1,2,0,494.0,0.0,247.0,247.0,247.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,17,2,2,0,458.0,0.0,229.0,229.0,229.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [ ]:
print(label_encoder.classes_)

NameError: name 'label_encoder' is not defined

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

# Selected features based on the data you shared
selected_features = [
    'Protocol',
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Fwd Packets Length Total',
    'Bwd Packets Length Total',
    'Fwd Packet Length Max',
    'Fwd Packet Length Min',
    'Fwd Packet Length Mean',
    'Fwd Packet Length Std'
]

target_column = 'Label'

# Load the dataset
data = pd.read_parquet("/content/data.parquet")
print("Data shape after loading:", data.shape)

# Encode the target variable
label_encoder = LabelEncoder()
data[target_column] = label_encoder.fit_transform(data[target_column])

# Split features and target
X = data[selected_features]
y = data[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(2, activation='relu', input_dim=X_train.shape[1]),

    Dense(2, activation='relu'),

    Dense(len(label_encoder.classes_), activation='softmax')  # Multi-class classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs=100, batch_size=64, verbose=2
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the trained model
model.save("ddos_attack_type_model.h5")

# Example usage: Predict the attack type
sample_prediction = model.predict(X_test[:1])
predicted_class = label_encoder.inverse_transform([sample_prediction.argmax()])
print(f"Predicted DDoS Attack Type: {predicted_class[0]}")

import struct
import numpy as np

def float_to_ieee754(value):
    """Convert a float to IEEE 754 single-precision (32-bit) representation."""
    packed = struct.pack('>f', value)  # Pack as big-endian float
    ieee754 = struct.unpack('>I', packed)[0]  # Unpack as unsigned int
    return f"0x{ieee754:08X}"  # Return as hexadecimal string

# Extract weights and biases from the trained model
with open("WeightsAndBiases_IEEE754.txt", "w") as weights_file:
    weights_file.write("Weights and Biases of the Trained Neural Network in IEEE 754 Format:\n\n")

    for i, layer in enumerate(model.layers):
        weights_file.write(f"Layer {i + 1}: {layer.name}\n")

        if hasattr(layer, "get_weights"):
            weights = layer.get_weights()

            if len(weights) > 0:
                weights_file.write("Weights (IEEE 754):\n")
                ieee_weights = np.vectorize(float_to_ieee754)(weights[0].flatten())
                weights_str = ", ".join(ieee_weights)
                weights_file.write("{" + weights_str + "},\n\n")

            if len(weights) > 1:
                weights_file.write("Biases (IEEE 754):\n")
                ieee_biases = np.vectorize(float_to_ieee754)(weights[1].flatten())
                biases_str = ", ".join(ieee_biases)
                weights_file.write("{" + biases_str + "},\n\n")
        else:
            weights_file.write("No weights available for this layer.\n\n")

print("Weights and biases in IEEE 754 format saved in 'WeightsAndBiases_IEEE754.txt'")


Data shape after loading: (431371, 78)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
5393/5393 - 16s - 3ms/step - accuracy: 0.5351 - loss: 1.4540 - val_accuracy: 0.6937 - val_loss: 1.0608
Epoch 2/100
5393/5393 - 10s - 2ms/step - accuracy: 0.6952 - loss: 1.0721 - val_accuracy: 0.6992 - val_loss: 0.9507
Epoch 3/100
5393/5393 - 9s - 2ms/step - accuracy: 0.6995 - loss: 1.0203 - val_accuracy: 0.7010 - val_loss: 0.9217
Epoch 4/100
5393/5393 - 10s - 2ms/step - accuracy: 0.7044 - loss: 1.0108 - val_accuracy: 0.7047 - val_loss: 0.8899
Epoch 5/100
5393/5393 - 11s - 2ms/step - accuracy: 0.7382 - loss: 0.8556 - val_accuracy: 0.7726 - val_loss: 0.5964
Epoch 6/100
5393/5393 - 19s - 3ms/step - accuracy: 0.7755 - loss: 0.6698 - val_accuracy: 0.7801 - val_loss: 0.5591
Epoch 7/100
5393/5393 - 9s - 2ms/step - accuracy: 0.7798 - loss: 0.6397 - val_accuracy: 0.7802 - val_loss: 0.5466
Epoch 8/100
5393/5393 - 10s - 2ms/step - accuracy: 0.7814 - loss: 0.6247 - val_accuracy: 0.7798 - val_loss: 0.5398
Epoch 9/100
5393/5393 - 11s - 2ms/step - accuracy: 0.7823 - loss: 0.6132 - val_acc

Test Accuracy: 78.81%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
Predicted DDoS Attack Type: DrDoS_NTP
Weights and biases in IEEE 754 format saved in 'WeightsAndBiases_IEEE754.txt'


In [ ]:
# Extract weights and biases from the trained model
with open("WeightsAndBiases.txt", "w") as weights_file:
    weights_file.write("Weights and Biases of the Trained Neural Network:\n\n")

    for i, layer in enumerate(model.layers):
        weights_file.write(f"Layer {i + 1}: {layer.name}\n")

        if hasattr(layer, "get_weights"):
            weights = layer.get_weights()

            if len(weights) > 0:
                # Write the weights
                weights_file.write("Weights:\n")
                scaled_weights = (weights[0] * 1e5).astype(int)  # Scaling by 1e5
                weights_str = np.array2string(
                    scaled_weights, threshold=np.inf, precision=4, separator=",",
                    suppress_small=True
                ).replace('[', '{').replace(']', '}')
                weights_file.write(weights_str + ",\n\n")

            if len(weights) > 1:
                # Write the biases
                weights_file.write("Biases:\n")
                scaled_biases = (weights[1] * 1e5).astype(int)  # Scaling by 1e5
                biases_str = np.array2string(
                    scaled_biases, threshold=np.inf, precision=4, separator=",",
                    suppress_small=True
                ).replace('[', '{').replace(']', '}')
                weights_file.write(biases_str + ",\n\n")
        else:
            weights_file.write("No weights available for this layer.\n\n")

print("Weights and biases saved in 'WeightsAndBiases.txt'")


Weights and biases saved in 'WeightsAndBiases.txt'


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the same model architecture
model = Sequential([
    Dense(128, activation='relu', input_dim=10),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(18, activation='softmax')  # Assuming 18 classes
])

# Initialize lists for weights and biases
weights = []
biases = []
current_block = []

# Parse weights and biases from the file
with open("WeightsAndBiases.txt", "r") as file:
    for line in file:
        if "Weights:" in line or "Biases:" in line:
            # Save the current block (if it's non-empty)
            if current_block:
                parsed_block = np.array(current_block)  # Convert to NumPy array
                if "Weights" in line:  # Last processed block was weights
                    weights.append(parsed_block)
                else:  # Last processed block was biases
                    biases.append(parsed_block)
            current_block = []  # Reset for the next block
        elif "{" in line and "}" in line:
            # Extract and parse the values
            values = line.strip("{} \n").split(",")
            current_block.append([int(v.strip()) / 1e5 for v in values])  # Scale back
    # Handle the last block
    if current_block:
        parsed_block = np.array(current_block)
        if "Weights" in line:  # If the last block was weights
            weights.append(parsed_block)
        else:  # If the last block was biases
            biases.append(parsed_block)

# Verify weights and biases counts
print(f"Number of weight sets: {len(weights)}")
print(f"Number of bias sets: {len(biases)}")

# Set the weights and biases into the model
for i, layer in enumerate(model.layers):
    if hasattr(layer, "set_weights"):
        try:
            # Ensure the correct shape by transposing weight matrices as needed
            weight = np.array(weights[i])
            bias = np.array(biases[i]).flatten()  # Biases must be 1D
            layer.set_weights([weight, bias])
            print(f"Set weights and biases for layer {layer.name}")
        except IndexError:
            print(f"Layer {layer.name} does not have weights/biases in the file.")
        except ValueError as e:
            print(f"Shape mismatch for layer {layer.name}: {e}")

print("Weights and biases loaded into the model.")


Number of weight sets: 0
Number of bias sets: 0
Layer dense_18 does not have weights/biases in the file.
Layer dropout_12 does not have weights/biases in the file.
Layer dense_19 does not have weights/biases in the file.
Layer dropout_13 does not have weights/biases in the file.
Layer dense_20 does not have weights/biases in the file.
Weights and biases loaded into the model.


In [ ]:
selected_features = [
    'Protocol',
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Fwd Packets Length Total',
    'Bwd Packets Length Total',
    'Fwd Packet Length Max',
    'Fwd Packet Length Min',
    'Fwd Packet Length Mean',
    'Fwd Packet Length Std'
]

# Extract the first five rows of the selected features
# subset = data[selected_features].head()

# Convert to IEEE 754 format
def to_ieee754(value):
    if isinstance(value, (float, int)):
        return hex(struct.unpack('>I', struct.pack('>f', float(value)))[0])
    return None  # Handle non-numeric values if needed

# Apply conversion to each element in the DataFrame
ieee754_converted = data.map(to_ieee754)

print(ieee754_converted)

     Protocol Flow Duration Total Fwd Packets Total Backward Packets  \
0  0x3f800000    0x4640e400        0x41200000             0x40a00000   
1  0x40c00000    0x47849900        0x41a00000             0x41200000   
2  0x41880000    0x46b74000        0x41f00000             0x41700000   
3  0x3f800000    0x479a1a80        0x42200000             0x41a00000   
4  0x40c00000    0x47070700        0x42480000             0x41c80000   

  Fwd Packets Length Total Bwd Packets Length Total Fwd Packet Length Max  \
0               0x42c90000               0x424a0000            0x42720000   
1               0x4348199a               0x42c86666            0x428d999a   
2               0x4396199a               0x43166666            0x42a1cccd   
3               0x43c86666               0x4348999a            0x42b40000   
4               0x43fa7333               0x437ab333            0x42c83333   

  Fwd Packet Length Min Fwd Packet Length Mean Fwd Packet Length Std  
0            0x3f99999a          

In [ ]:
# |
import numpy as np

# Adjust print options to avoid truncation
np.set_printoptions(suppress=True, precision=4, linewidth=200, threshold=np.inf)

print(weights)

[array([[-0.3491,  0.0369,  0.0865,  0.0217,  0.1834, -0.1087,  0.0864, -0.5011,  0.0845, -0.0332, -0.782 , -0.4631, -0.3584,  0.0088, -0.4618, -0.7453, -0.5147, -0.2046],
       [-1.0478, -0.1065, -0.5998,  0.1845,  0.1221, -0.3713, -0.4991,  0.0164, -0.2199,  0.1819, -1.0469, -0.3704, -0.1713,  0.0933,  0.0514, -0.1899, -0.2513, -0.618 ],
       [-0.4182,  0.1984, -0.8307, -0.2332,  0.2469, -1.144 , -0.8916,  0.2132, -0.3485, -0.3525, -0.7497, -1.3718, -0.2654, -0.3566,  0.1953,  0.2952, -0.0862, -0.0561],
       [-0.3866, -0.0419,  0.0589, -0.3247,  0.5423, -0.1406, -0.4721, -0.5807,  0.1348, -0.3308, -0.4544, -0.0895, -0.324 ,  0.6479, -0.5728, -1.2683, -0.1213,  0.0251],
       [-0.3987, -0.1767, -0.3523, -0.4398,  0.8166, -0.0384, -0.641 , -0.4884, -0.6797, -0.3891, -0.5669, -0.0301, -0.5367,  0.77  , -0.5711, -0.9661, -0.4149, -0.18  ],
       [-0.4154,  0.0381, -0.8283, -0.6211,  0.1539,  0.192 , -0.1835,  0.0721,  0.0607, -0.5627,  0.1588,  0.1666, -0.1219, -1.3176,  0.0834,  

In [ ]:
with open("WeightsAndBiases.txt", "w") as weights_file:
    weights_file.write("Weights and Biases of the Trained Neural Network:\n\n")

    for i, layer in enumerate(model.layers):
        if hasattr(layer, "weights"):  # Check if the layer has weights
            weights = layer.get_weights()

            if weights:
                weights_file.write(f"Layer {i + 1}: {layer.name}\n")

                if len(weights) > 0:
                    weights_file.write("Weights:\n")
                    weights_file.write(str(weights[0]))
                    weights_file.write("\n\n")

                if len(weights) > 1:
                    weights_file.write("Biases:\n")
                    weights_file.write(str(weights[1]))
                    weights_file.write("\n\n")

print("Weights and biases saved in 'WeightsAndBiases.txt'")

Weights and biases saved in 'WeightsAndBiases.txt'


In [ ]:
len(le_target.classes_)

In [ ]:
import numpy as np

def quantize_to_int16(weights, min_val=None, max_val=None):
    """
    Quantizes weights to 16-bit signed integers and returns their hexadecimal representation.

    Parameters:
        weights (numpy array): The weights to quantize.
        min_val (float): Minimum value for scaling. If None, the minimum of weights is used.
        max_val (float): Maximum value for scaling. If None, the maximum of weights is used.

    Returns:
        hex_quantized_weights (list of str): List of hexadecimal strings representing quantized weights.
        scale (float): Scaling factor used for quantization.
        zero_point (int): Zero point for quantization.
    """
    # Determine the range of weights
    min_val = weights.min() if min_val is None else min_val
    max_val = weights.max() if max_val is None else max_val

    # Ensure the range is valid
    if min_val == max_val:
        raise ValueError("Min and max values must not be the same.")

    # Define the quantization range for int16
    quant_min = -32768  # Minimum for int16
    quant_max = 32767   # Maximum for int16

    # Calculate scale and zero point
    scale = (max_val - min_val) / (quant_max - quant_min)
    zero_point = int(quant_min - min_val / scale)

    # Quantize the weights
    quantized_weights = (weights / scale + zero_point).round().astype(np.int16)

    # Clip the quantized values to the 16-bit range to prevent overflow
    quantized_weights = np.clip(quantized_weights, quant_min, quant_max)

    # Convert quantized weights to hexadecimal representation
    hex_quantized_weights = [f"{str(hex(w))[3:]}" for w in quantized_weights]

    return hex_quantized_weights, scale, zero_point


def to_hex(value):
    """Convert a signed 16-bit integer to hexadecimal."""
    return f"{value & 0xFFFF:04X}"


# Zero-point quantization parameters
scaling_factor = 0.1 # Chosen scale factor (can be adjusted based on the model's range)
zero_point = 128  # Chosen zero point for asymmetric quantization (can be adjusted)

# Assuming `model` is your trained neural network model
weights = model.get_weights()[0::2]  # Extract weights (even indices)
biases = model.get_weights()[1::2]  # Extract biases (odd indices)

# Quantize weights and biases using zero-point quantization
quantized_weights = [
    [[quantize_to_int16(np.array(neuron_weights), scaling_factor)[0] for neuron_weights in layer_weights]]
    for layer_weights in weights
]

quantized_biases = [
    [quantize_to_int16(np.array(layer_biases), scaling_factor)[0] for layer_biases in biases]
]

# Generate Verilog parameters for weights and biases
verilog_parameters = []

# Weight parameters
for layer_idx, layer_weights in enumerate(quantized_weights):
    for neuron_idx, neuron_weights in enumerate(layer_weights):
        for weight_idx, weight in enumerate(neuron_weights):
            verilog_parameters.append(
                f"parameter [15:0] w{layer_idx+1}{neuron_idx+1}_{weight_idx+1} = 16'h{weight[0]};"
            )

print(quantized_biases)
# Bias parameters
for layer_idx, layer_biases in enumerate(quantized_biases):
    for neuron_idx, bias in enumerate(layer_biases):
        verilog_parameters.append(
            f"parameter [15:0] b{layer_idx+1}_{neuron_idx+1} = 16'h{bias[0]};"
        )

# Save Verilog parameters to a file
with open("nn_parameters_zero_point.v", "w") as f:
    f.write("\n".join(verilog_parameters))

print("Zero-point quantized Verilog parameters have been saved to 'nn_parameters_zero_point.vh'.")

# Example usage: Predict the attack type (assuming model and data are ready)
sample_prediction = model.predict(X_test[:1])  # Predict for the first sample in the test set
predicted_class = le_target.inverse_transform([np.argmax(sample_prediction)])
print(f"Predicted DDoS Attack Type: {predicted_class[0]}")


[[['905', 'ec4', '18a', '4cc4', '67c', '4a03', '89f', '79f', 'a9a', '612', '17c3', '4e3e', 'b4a', 'f6', '515f', 'd5b', '6ca6', '376c', 'ae', '742d', '7fa', '7e1d', '7e57', 'ecb', 'dc6', '2107', 'fff', '5f5a', '2100', 'c52', 'f41', '1be', '3bc8', '0c3', '49', 'd04', '6f5', '7145', '0c2', '3ecb', 'f98', 'ca3', '557', '706d', 'e8f', '60b1', '85', '6155', '7e6', 'a9', '75', '186c', '507a', '127', '89e', '548a', '00c', '51fd', '7a88', '5e86', 'b9b', '2aaa', '4cfd', '50b', '1ec9', '709d', '161a', '5b96', '71b2', '377', '27c5', '7c0', '106', 'b3a', '626e', 'dbd', '2f5', '21c8', '1b8f', 'f7', '61a', 'e0a', '3e2', '216', '6931', 'fd7', '2', 'ed1', '23', 'd11', '618f', '52c7', '62af', 'a07', 'b19', '6f23', 'c30', '1a5', '0fa', '119', '27a', '4862', '7a7', 'bb4', '3ca2', '1bcb', '3ae', '629', '422f', '7a', 'b49', '631f', '23', '253a', '813', '0ff', '649', '6694', 'cea', '1d6', '1eda', '6ae5', '7a12', '526', '6fd', '7f1c', '11d6', '2d9'], ['588', 'adf', '6ac3', '702d', '706', '1a61', 'e8a', '3af5'

In [ ]:
le_target.classes_

array(['Benign', 'DrDoS_DNS', 'DrDoS_LDAP', 'DrDoS_MSSQL', 'DrDoS_NTP',
       'DrDoS_NetBIOS', 'DrDoS_SNMP', 'DrDoS_UDP', 'LDAP', 'MSSQL',
       'NetBIOS', 'Portmap', 'Syn', 'TFTP', 'UDP', 'UDP-lag', 'UDPLag',
       'WebDDoS'], dtype=object)

In [ ]:
print(sample_prediction)

[[4.19882526e-05 2.43530981e-02 4.04087950e-05 3.03742707e-01
  1.03227526e-01 6.38544699e-03 3.06867034e-04 1.66013651e-02
  2.65607378e-04 5.02128839e-01 1.17157042e-05 4.72763146e-04
  1.10186310e-03 1.23460370e-03 3.74713652e-02 2.12648069e-03
  4.85996483e-04 1.37321570e-06]]


In [ ]:
# prompt: using above weights predict output from quantized values

import numpy as np

# Load the quantized weights and bias

def predict_with_quantized_weights(input_data, weights, biases):
    """Simulate forward pass using quantized weights and biases."""

    # Reshape input data if necessary
    input_data = np.array(input_data).reshape(1, -1)

    activations = [input_data]
    for layer in range(1, len(weights) + 1):
        layer_output = []
        for neuron in range(1, len(weights[layer]) + 1):
            weighted_sum = biases[layer][neuron] / scaling_factor
            for weight_idx, weight in weights[layer][neuron].items():
                 weighted_sum += (weight / scaling_factor) * activations[-1][0][weight_idx-1]

            # Apply ReLU activation for hidden layers
            if layer < len(weights):
                output = max(0, weighted_sum)
            else:  # Softmax for output layer (simplified for demonstration)
                output = weighted_sum
            layer_output.append(output)
        activations.append(np.array([layer_output]))

    return activations[-1]


# Example Usage:
quantized_weights, quantized_biases = load_quantized_parameters()
# Replace this with your actual input data
input_sample = X_test[:1].tolist()[0]

quantized_prediction = predict_with_quantized_weights(input_sample, quantized_weights, quantized_biases)
print("Predicted Output (Quantized):", np.argmax(quantized_prediction))
# ... (Rest of your code)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# Load the weights and biases from the Verilog file
quantized_weights, quantized_biases = load_quantized_parameters("nn_parameters_zero_point.vh")

# Define the scale and zero-point used during quantization
scale = 0.05  # Adjust this based on your quantization settings
zero_point = 128  # Adjust this based on your quantization settings

# Example input data (replace with actual test input)
input_sample = np.random.uniform(-1, 1, size=10).tolist()  # Adjust size to match input dimensions

# Predict using the quantized parameters
quantized_prediction = predict_with_zero_point_quantized_weights(
    input_sample, quantized_weights, quantized_biases, scale, zero_point
)

# Print the raw output and predicted class
print("Predicted Output (Quantized):", quantized_prediction)
predicted_class = np.argmax(quantized_prediction)
print("Predicted Class:", predicted_class)


UnboundLocalError: local variable 'neuron_idx' referenced before assignment

In [ ]:
# prompt: ValueError                                Traceback (most recent call last)
# <ipython-input-33-51f688092501> in <cell line: 2>()
#       1 # Load the weights and biases from the Verilog file
# ----> 2 quantized_weights, quantized_biases = load_quantized_parameters("nn_parameters_zero_point.vh")
#       3
#       4 # Define the scale and zero-point used during quantization
#       5 scale = 0.05  # Adjust this based on your quantization settings
# <ipython-input-32-0d9c3dc8a40d> in load_quantized_parameters(filename)
#      30                     hex_value = hex_value.replace(";", "")
#      31
# ---> 32                     int_value = int(hex_value, 16) # Changed from base 8 to base 16
#      33
#      34                     if param_name.startswith("w"):
# ValueError: invalid literal for int() with base 16: "8'h85"

import re

def load_quantized_parameters(filename):
    """Loads quantized weights and biases from a Verilog file."""
    weights = {}
    biases = {}
    with open(filename, 'r') as f:
        for line in f:
            match = re.match(r"parameter \[7:0] (\w+)(\d+)_?(\d+)? = 8'h([0-9a-fA-F]+);", line)
            if match:
                param_name = match.group(1)
                layer_idx = int(match.group(2))
                if match.group(3):
                  neuron_idx = int(match.group(3))
                hex_value = match.group(4)
                int_value = int(hex_value, 16)

                if param_name.startswith("w"):
                    if layer_idx not in weights:
                        weights[layer_idx] = {}
                    if neuron_idx not in weights[layer_idx]:
                        weights[layer_idx][neuron_idx] = {}
                    weights[layer_idx][neuron_idx][int(match.group(3))] = int_value
                elif param_name.startswith("b"):
                    if layer_idx not in biases:
                        biases[layer_idx] = {}
                    biases[layer_idx][neuron_idx] = int_value
    return weights, biases

In [ ]:
!pip install hls4ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninst

In [ ]:
import hls4ml

QKeras installation not found, installing one...
QKeras installed.


ModuleNotFoundError: No module named 'tensorflow.keras.utils'

In [ ]:
# Convert Keras model to HLS model
hls_model = hls4ml.convert_from_keras(model, hls_config='hls4ml_config.yaml')

# Compile the HLS model
hls_model.compile()


NameError: name 'hls4ml' is not defined

In [ ]:
from typing import List
import numpy as np
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd

class Model:
    def __init__(self, model: nn.Sequential):
        self.model = model
        self.layers = []

    def __str__(self):
        return '\n'.join(str(layer) for layer in self.layers)

    def parse_layers(self):
        for i, layer in enumerate(self.model):
            if isinstance(layer, nn.Linear):
                self.layers.append(layer.state_dict()['weight'])
                self.layers.append(layer.state_dict()['bias'])
            elif isinstance(layer, nn.ReLU):
                self.layers.append(nn.ReLU(self.model[i - 1].out_features, i))
            else:
                raise ValueError(f"Unknown layer type {layer}")

    def forward(self, x: torch.Tensor):
        return self.model(x)

    def forward_range(self, ranges: List[List[float]]):
        start = np.array(ranges)

        for layer in self.layers:
            start = layer.forward_range(start)

    def get_vars(self):
        in_params = [f"in{i}" for i in range(self.layers[0].shape[0])]
        out_params = [f"out{i}" for i in range(self.layers[-1].shape[-1])]

        in_definitions = [f"    input [{self.layers[0].in_bits[i] - 1}:0] {in_params[i]};"
                          for i in range(self.layers[0].shape[0])]

        out_definitions = [f"    output [{self.layers[-1].out_bits[i] - 1}:0] {out_params[i]};"
                           for i in range(self.layers[-1].shape[-1])]

        return in_params, out_params, in_definitions, out_definitions

    def emit(self):
        out = ["`timescale 1ns / 1ps"]

        in_params, out_params, in_definitions, out_definitions = self.get_vars()

        top = [
            f"module top({','.join(in_params)}, {','.join(out_params)});",
            *in_definitions,
            *out_definitions,
        ]

        in_wires = in_params
        out_wires = []

        for i, layer in enumerate(self.layers):
            out.append(layer.emit())

            out_wires = []
            for j in range(layer.shape[-1]):
                top.append(f"    wire [{layer.out_bits[j]}:0] layer_{i}_out_{j};")
                out_wires.append(f"layer_{i}_out_{j}")

            top.append(f"    {layer.name} layer_{i}({','.join(in_wires)}, {','.join(out_wires)});")

            in_wires = out_wires

        assigns = [f"    assign out{i} = {out_wire};" for i, out_wire in enumerate(out_wires)]

        top.extend(assigns)
        top.append("endmodule")

        out.append('\n'.join(top))

        return '\n'.join(out)


def train_model():
    # Load the dataset
    data = pd.read_parquet("/content/data.parquet")

    # Feature selection and preprocessing
    selected_features = [
        "Protocol", "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
        "Fwd Packets Length Total", "Bwd Packets Length Total", "Fwd Packet Length Max",
        "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std"
    ]
    target_column = "Label"

    le_target = LabelEncoder()
    data[target_column] = le_target.fit_transform(data[target_column])

    X = data[selected_features]
    y = data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define the PyTorch model
    model = nn.Sequential(
        nn.Linear(len(selected_features), 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64, len(le_target.classes_)),
    )

    # Training loop
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10
    batch_size = 32

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predictions = torch.max(outputs, 1)
        accuracy = (predictions == y_test_tensor).float().mean()
        print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

    # Save the model
    torch.save(model.state_dict(), "ddos_attack_type_model.pt")
    print("Model saved!")

if __name__ == "__main__":
    train_model()


Epoch [1/10], Loss: 2.8951
Epoch [2/10], Loss: 2.8618
Epoch [3/10], Loss: 2.8282
Epoch [4/10], Loss: 2.7942
Epoch [5/10], Loss: 2.7604
Epoch [6/10], Loss: 2.7261
Epoch [7/10], Loss: 2.6908
Epoch [8/10], Loss: 2.6547
Epoch [9/10], Loss: 2.6171
Epoch [10/10], Loss: 2.5781
Test Accuracy: 55.04%
Model saved!


In [ ]:
import torch
from torch import nn
# import layers
# Define a simple PyTorch model
simple_model = nn.Sequential(
        nn.Linear(9, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64, 18),
    )

# Wrap the model in the custom Model class
model = Model(simple_model)

# Parse the layers to prepare for Verilog generation
model.parse_layers()

# Generate Verilog code
verilog_code = model.emit()

# Save the Verilog code to a file
with open("generated_model.v", "w") as f:
    f.write(verilog_code)

print("Verilog code has been generated and saved as 'generated_model.v'.")


AttributeError: 'Linear' object has no attribute 'Linear'

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
# Selected features based on the data you shared
selected_features = [
    'Protocol',
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Fwd Packets Length Total',
    'Bwd Packets Length Total',
    'Fwd Packet Length Max',
    'Fwd Packet Length Min',
    'Fwd Packet Length Mean',
    'Fwd Packet Length Std'
]

target_column = 'Label'

data = pd.read_parquet("/content/data.parquet")
# Check the data structure and clean if necessary
print("Data shape after concatenation:", data.shape)

# Encode the target variable
le_target = LabelEncoder()
data[target_column] = le_target.fit_transform(data[target_column])

# Split features and target
X = data[selected_features]
y = data[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
nn.fit(X_train, y_train)

y_pred = nn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)



weights = nn.coefs_
biases = nn.intercepts_

def quantize_to_fixed_point(value, scaling_factor=2048):
    """Quantize a float value to a fixed-point 16-bit integer and clamp to the 16-bit range."""
    fixed_point_value = int(value * scaling_factor)
    if fixed_point_value < -32768:
        fixed_point_value = -32768
    elif fixed_point_value > 32767:
        fixed_point_value = 32767
    return fixed_point_value

def to_hex(value):
    """Convert a signed 16-bit integer to hexadecimal."""
    return f"{value & 0xFFFF:04X}"


quantized_weights = [
    [
        [quantize_to_fixed_point(weight, scaling_factor=2048) for weight in neuron_weights]
        for neuron_weights in layer_weights
    ]
    for layer_weights in weights
]

quantized_biases = [
    [quantize_to_fixed_point(bias, scaling_factor=2048) for bias in layer_biases]
    for layer_biases in biases
]

verilog_parameters = []

for layer_idx, layer_weights in enumerate(quantized_weights):
    for neuron_idx, neuron_weights in enumerate(layer_weights):
        for weight_idx, weight in enumerate(neuron_weights):
            verilog_parameters.append(
                f"parameter signed [15:0] w{layer_idx+1}{neuron_idx+1}_{weight_idx+1} = 16'h{to_hex(weight)};"
            )


for layer_idx, layer_biases in enumerate(quantized_biases):
    for neuron_idx, bias in enumerate(layer_biases):
        verilog_parameters.append(
            f"parameter signed [15:0] b{layer_idx+1}_{neuron_idx+1} = 16'h{to_hex(bias)};"
        )


print("\n".join(verilog_parameters))


FileNotFoundError: [Errno 2] No such file or directory: '/content/data.parquet'

In [ ]:
print(accuracy)

NameError: name 'accuracy' is not defined

In [ ]:
import pandas as pd
import numpy as np

def quantize_and_convert_to_hex(value, min_val, max_val):
    """Quantize the value and convert it to 16-bit signed hexadecimal."""
    # Avoid division by zero if min_val == max_val
    if min_val == max_val:
        # If all values are the same, return a fixed hexadecimal value
        quantized = 0  # or you could set this to any valid value within the range
    else:
        # Normalize to the range [0, 1]
        quantized = int(2048 * (value - min_val) / (max_val - min_val))

    # Clamp to 16-bit signed range
    if quantized < -32768:
        quantized = -32768
    elif quantized > 32767:
        quantized = 32767

    # Convert to hexadecimal
    return f"{quantized & 0xFFFF:04X}"

# Load the parquet file
data = pd.read_parquet('/content/data.parquet')

# Select the first 5 rows
data = data.head(5)

# Define columns to process
columns_to_process = [
    'Protocol',
    'Flow Duration',
    'Total Fwd Packets',
    'Total Backward Packets',
    'Fwd Packets Length Total',
    'Bwd Packets Length Total',
    'Fwd Packet Length Max',
    'Fwd Packet Length Min',
    'Fwd Packet Length Mean',
    'Fwd Packet Length Std'
]

# Iterate through each column
for col in columns_to_process:
    # Normalizing the column
    values = data[col]
    min_value = min(values)
    max_value = max(values)

    # Generate hex values for the column
    hex_values = [quantize_and_convert_to_hex(val, min_value, max_value) for val in values]

    # Print results for the first 5 rows
    for i, hex_val in enumerate(hex_values):
        print(f"Row {i + 1}: {col} = {data[col].iloc[i]}, Hexadecimal = {hex_val}")


Row 1: Protocol = 6, Hexadecimal = 0000
Row 2: Protocol = 6, Hexadecimal = 0000
Row 3: Protocol = 6, Hexadecimal = 0000
Row 4: Protocol = 6, Hexadecimal = 0000
Row 5: Protocol = 6, Hexadecimal = 0000
Row 1: Flow Duration = 105, Hexadecimal = 0000
Row 2: Flow Duration = 1, Hexadecimal = 0000
Row 3: Flow Duration = 55295858, Hexadecimal = 0800
Row 4: Flow Duration = 49, Hexadecimal = 0000
Row 5: Flow Duration = 109, Hexadecimal = 0000
Row 1: Total Fwd Packets = 2, Hexadecimal = 0000
Row 2: Total Fwd Packets = 2, Hexadecimal = 0000
Row 3: Total Fwd Packets = 8, Hexadecimal = 0800
Row 4: Total Fwd Packets = 2, Hexadecimal = 0000
Row 5: Total Fwd Packets = 2, Hexadecimal = 0000
Row 1: Total Backward Packets = 2, Hexadecimal = 0800
Row 2: Total Backward Packets = 0, Hexadecimal = 0000
Row 3: Total Backward Packets = 0, Hexadecimal = 0000
Row 4: Total Backward Packets = 0, Hexadecimal = 0000
Row 5: Total Backward Packets = 2, Hexadecimal = 0800
Row 1: Fwd Packets Length Total = 12.0, Hexadeci

In [ ]:

# module layer9x18 (
#     input signed [15:0] in1,
#     input signed [15:0] in2,
#     input signed [15:0] in3,
#     input signed [15:0] in4,
#     input signed [15:0] in5,
#     input signed [15:0] in6,
#     input signed [15:0] in7,
#     input signed [15:0] in8,
#     input signed [15:0] in9,
#     output signed [15:0] out1,
#     output signed [15:0] out2,
#     output signed [15:0] out3,
#     output signed [15:0] out4,
#     output signed [15:0] out5,
#     output signed [15:0] out6,
#     output signed [15:0] out7,
#     output signed [15:0] out8,
#     output signed [15:0] out9,
#     output signed [15:0] out10,
#     output signed [15:0] out11,
#     output signed [15:0] out12,
#     output signed [15:0] out13,
#     output signed [15:0] out14,
#     output signed [15:0] out15,
#     output signed [15:0] out16,
#     output signed [15:0] out17,
#     output signed [15:0] out18
# );

# // Declare parameters (weights and biases)
# parameter signed [15:0] w11_1 = 16'h046C, w11_2 = 16'h19A5;
# parameter signed [15:0] w12_1 = 16'hF5A9, w12_2 = 16'hEA35;
# parameter signed [15:0] w13_1 = 16'h048C, w13_2 = 16'h281E;
# parameter signed [15:0] w14_1 = 16'h0B22, w14_2 = 16'h15AE;
# parameter signed [15:0] w15_1 = 16'h3EF8, w15_2 = 16'h38EB;
# parameter signed [15:0] w16_1 = 16'hFFE2, w16_2 = 16'h0F95;
# parameter signed [15:0] w17_1 = 16'hD939, w17_2 = 16'h0850;
# parameter signed [15:0] w18_1 = 16'hC026, w18_2 = 16'hD273;
# parameter signed [15:0] w19_1 = 16'hB73F, w19_2 = 16'h0491;
# parameter signed [15:0] w110_1 = 16'h2420, w110_2 = 16'h3396;
# parameter signed [15:0] w21_1 = 16'h1A02, w21_2 = 16'hFF0D, w21_3 = 16'hE001;
# parameter signed [15:0] w21_4 = 16'hD801, w21_5 = 16'hF4CD, w21_6 = 16'h05A6;
# parameter signed [15:0] w21_7 = 16'hEAC8, w21_8 = 16'h024B, w21_9 = 16'hDCB5;
# parameter signed [15:0] w21_10 = 16'hD8AD, w21_11 = 16'h089A, w21_12 = 16'h09D9;
# parameter signed [15:0] w21_13 = 16'h1D6D, w21_14 = 16'h9D4B, w21_15 = 16'h0158;
# parameter signed [15:0] w21_16 = 16'h0FD9, w21_17 = 16'hFBDF, w21_18 = 16'h033F;
# parameter signed [15:0] w22_1 = 16'hF48D, w22_2 = 16'hF261, w22_3 = 16'hE927;
# parameter signed [15:0] w22_4 = 16'h05F7, w22_5 = 16'h1015, w22_6 = 16'hF9C9;
# parameter signed [15:0] w22_7 = 16'hEDC9, w22_8 = 16'h01AC, w22_9 = 16'hEC9D;
# parameter signed [15:0] w22_10 = 16'h0590, w22_11 = 16'hFAA5, w22_12 = 16'hF73F;
# parameter signed [15:0] w22_13 = 16'hDD2D, w22_14 = 16'h1629, w22_15 = 16'h0414;
# parameter signed [15:0] w22_16 = 16'hEFD7, w22_17 = 16'hF4B6, w22_18 = 16'hF07B;

# parameter signed [15:0] b1_1 = 16'h185D, b1_2 = 16'hFF97;
# parameter signed [15:0] b2_1 = 16'hF424, b2_2 = 16'h121C, b2_3 = 16'h10E4;
# parameter signed [15:0] b2_4 = 16'h0742, b2_5 = 16'hF9B5, b2_6 = 16'hF24A;
# parameter signed [15:0] b2_7 = 16'h1599, b2_8 = 16'hFA24, b2_9 = 16'h130E;
# parameter signed [15:0] b2_10 = 16'h0A80, b2_11 = 16'hED23, b2_12 = 16'hF359;
# parameter signed [15:0] b2_13 = 16'hF78A, b2_14 = 16'h070D, b2_15 = 16'hF9E8;
# parameter signed [15:0] b2_16 = 16'h04AF, b2_17 = 16'hF1D7, b2_18 = 16'hF234;

# // Implement weighted sums with bias for each output
# assign out1  = (in1 * w11_1 + in2 * w11_2 + in3 * w12_1 + in4 * w12_2 + in5 * w13_1 + in6 * w13_2 + in7 * w14_1 + in8 * w14_2 + in9 * w15_1) >>> 8 + b1_1;
# assign out2  = (in1 * w16_1 + in2 * w16_2 + in3 * w17_1 + in4 * w17_2 + in5 * w18_1 + in6 * w18_2 + in7 * w19_1 + in8 * w19_2 + in9 * w110_1) >>> 8 + b1_2;
# assign out3  = (in1 * w110_2 + in2 * w21_1 + in3 * w21_2 + in4 * w21_3 + in5 * w21_4 + in6 * w21_5 + in7 * w21_6 + in8 * w21_7 + in9 * w21_8) >>> 8 + b2_1;
# assign out4  = (in1 * w21_9 + in2 * w21_10 + in3 * w21_11 + in4 * w21_12 + in5 * w21_13 + in6 * w21_14 + in7 * w21_15 + in8 * w21_16 + in9 * w21_17) >>> 8 + b2_2;
# assign out5  = (in1 * w21_18 + in2 * w22_1 + in3 * w22_2 + in4 * w22_3 + in5 * w22_4 + in6 * w22_5 + in7 * w22_6 + in8 * w22_7 + in9 * w22_8) >>> 8 + b2_3;
# assign out6  = (in1 * w22_9 + in2 * w22_10 + in3 * w22_11 + in4 * w22_12 + in5 * w22_13 + in6 * w22_14 + in7 * w22_15 + in8 * w22_16 + in9 * w22_17) >>> 8 + b2_4;
# assign out7  = (in1 * w22_18 + in2 * w11_1 + in3 * w11_2 + in4 * w12_1 + in5 * w12_2 + in6 * w13_1 + in7 * w13_2 + in8 * w14_1 + in9 * w14_2) >>> 8 + b2_5;
# assign out8  = (in1 * w15_1 + in2 * w15_2 + in3 * w16_1 + in4 * w16_2 + in5 * w17_1 + in6 * w17_2 + in7 * w18_1 + in8 * w18_2 + in9 * w19_1) >>> 8 + b2_6;
# assign out9  = (in1 * w19_2 + in2 * w110_1 + in3 * w110_2 + in4 * w21_1 + in5 * w21_2 + in6 * w21_3 + in7 * w21_4 + in8 * w21_5 + in9 * w21_6) >>> 8 + b2_7;
# assign out10 = (in1 * w21_7 + in2 * w21_8 + in3 * w21_9 + in4 * w21_10 + in5 * w21_11 + in6 * w21_12 + in7 * w21_13 + in8 * w21_14 + in9 * w21_15) >>> 8 + b2_8;
# assign out11 = (in1 * w21_16 + in2 * w21_17 + in3 * w21_18 + in4 * w22_1 + in5 * w22_2 + in6 * w22_3 + in7 * w22_4 + in8 * w22_5 + in9 * w22_6) >>> 8 + b2_9;
# assign out12 = (in1 * w22_7 + in2 * w22_8 + in3 * w22_9 + in4 * w22_10 + in5 * w22_11 + in6 * w22_12 + in7 * w22_13 + in8 * w22_14 + in9 * w22_15) >>> 8 + b2_10;
# assign out13 = (in1 * w22_16 + in2 * w22_17 + in3 * w22_18 + in4 * w11_1 + in5 * w11_2 + in6 * w12_1 + in7 * w12_2 + in8 * w13_1 + in9 * w13_2) >>> 8 + b2_11;
# assign out14 = (in1 * w14_1 + in2 * w14_2 + in3 * w15_1 + in4 * w15_2 + in5 * w16_1 + in6 * w16_2 + in7 * w17_1 + in8 * w17_2 + in9 * w18_1) >>> 8 + b2_12;
# assign out15 = (in1 * w18_2 + in2 * w19_1 + in3 * w19_2 + in4 * w110_1 + in5 * w110_2 + in6 * w21_1 + in7 * w21_2 + in8 * w21_3 + in9 * w21_4) >>> 8 + b2_13;
# assign out16 = (in1 * w21_5 + in2 * w21_6 + in3 * w21_7 + in4 * w21_8 + in5 * w21_9 + in6 * w21_10 + in7 * w21_11 + in8 * w21_12 + in9 * w21_13) >>> 8 + b2_14;
# assign out17 = (in1 * w21_14 + in2 * w21_15 + in3 * w21_16 + in4 * w21_17 + in5 * w21_18 + in6 * w22_1 + in7 * w22_2 + in8 * w22_3 + in9 * w22_4) >>> 8 + b2_15;
# assign out18 = (in1 * w22_5 + in2 * w22_6 + in3 * w22_7 + in4 * w22_8 + in5 * w22_9 + in6 * w22_10 + in7 * w22_11 + in8 * w22_12 + in9 * w22_13) >>> 8 + b2_16;

# endmodule